# ログ一括チェック
## 目的
- ルータの状態確認コマンド実行し、確認観点に従ってチェックするためのノートブック

# モジュールのインポート

In [1]:
import os
import sys
import json
from getpass import getpass
from datetime import datetime
from IPython.display import display, HTML

# --- 1. パス・ディレクトリ設定 ---
BASE_DIR = os.getcwd()
sys.path.append(os.path.join(BASE_DIR, "lib"))
RESULT_DIR = os.path.join(BASE_DIR, "result")
# 結果保存用のフォルダがない場合は作成
os.makedirs(RESULT_DIR, exist_ok=True)

# --- 2. 独自ライブラリの読み込み ---
from login_utils import login_telnet, login_jump_ssh
from exe_utils import load_commands, execute_commands, generate_html_report

# --- 3. 実行対象（ターゲット）と踏み台の指定 ---
target_id = "clab-dev-vmx01"
jump_id   = "clab-shuttle01"

# --- 4. デバイス情報の読み込み ---
with open(os.path.join(BASE_DIR, "vault", "device.json"), "r", encoding="utf-8") as f:
    device_data = json.load(f)

target_info = device_data[target_id]
jump_params = device_data[jump_id]

# --- 5. セキュリティ設定（パスワードの対話入力） ---
# JSONにパスワードが未記載の場合、実行時に入力プロンプトを表示する
if not jump_params.get("password"):
    jump_params["password"] = getpass(f"Password for Jump({jump_params['username']}): ")
if not target_info.get("password"):
    target_info["password"] = getpass(f"Password for Target({target_info['username']}): ")

# --- 6. 読み込みファイル・出力ファイルのパス定義 ---
CMD_FILE = os.path.join(BASE_DIR, "param", "commands.txt")

# 日付付きでログファイルとレポートファイルの名前を生成
timestamp = datetime.now().strftime('%Y%m%d')
LOG_FILE = os.path.join(RESULT_DIR, f"evid_{target_id}_{timestamp}.log")
REPORT_FILE = os.path.join(RESULT_DIR, f"report_{target_id}_{timestamp}.html")

# --- 7. セットアップ状況の確認表示 ---
print("-" * 50)
print(f"[{datetime.now().strftime('%H:%M:%S')}] [INFO]: Environment Setup Completed.")
print(f"  - Target ID  : {target_id}")
print(f"  - Target IP  : {target_info['host']} ({target_info['device_type']})")
print(f"  - Via Jump   : {jump_id} ({jump_params['host']})")
print(f"  - Result Dir : {RESULT_DIR}")
print("-" * 50)

--------------------------------------------------
[15:24:14] [INFO]: Environment Setup Completed.
  - Target ID  : clab-dev-vmx01
  - Target IP  : 172.20.20.3 (juniper_junos)
  - Via Jump   : clab-shuttle01 (192.168.3.23)
  - Result Dir : /home/jupyter_projects/show-cmd-checker/result
--------------------------------------------------


# 設定読み込みとログイン実行

In [2]:
try:
    print("-" * 50)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [INFO]: Starting connection sequence...")
    
    # --- 1. 踏み台へのログイン ---
    # まずは中継地点となる踏み台サーバへTelnetで接続。
    # ここでの出力は全て LOG_FILE に記録される。
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [STEP 1]: Logging into Jump-Host...")
    print(f"  - Host: {jump_params['host']}")
    
    conn = login_telnet(jump_params, "#", LOG_FILE) 
    print(f"  - Status: Jump-Host login successful.")

    # --- 2. ターゲットへのログイン（SSH Jump） ---
    # 踏み台のプロンプト上でSSHコマンドを発行し、目的のネットワーク機器へ移動。
    # login_jump_ssh内部で「redispatch」が行われ、制御モードがターゲットOSに切り替わる。
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [STEP 2]: Jumping to Target Device...")
    print(f"  - Host: {target_info['host']} ({target_info['device_type']})")
    
    conn = login_jump_ssh(
        conn, 
        target_info["host"], 
        target_info["username"], 
        target_info["password"], 
        target_info["device_type"]
    )
    
    # --- 3. ログイン先プロンプトの最終確認 ---
    # 最終的に到達した機器のプロンプトを取得し、意図した機器にログインできているかを確認。
    current_prompt = conn.find_prompt()
    
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [SUCCESS]: Reached Target.")
    print(f"  - Final Prompt: {current_prompt}")
    print("-" * 50)

except Exception as e:
    # ログイン工程のどこかで失敗した場合の例外処理
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] [ERROR]: Connection failed during login.")
    print(f"  - Details: {e}")
    print("-" * 50)
    
    # セッションが残っている場合は切断してリソースを解放
    if 'conn' in locals(): 
        conn.disconnect()
    # セルを停止させるためにエラーを再送
    raise e

--------------------------------------------------
[15:24:14] [INFO]: Starting connection sequence...
[15:24:14] [STEP 1]: Logging into Jump-Host...
  - Host: 192.168.3.23
  - Status: Jump-Host login successful.
[15:24:27] [STEP 2]: Jumping to Target Device...
  - Host: 172.20.20.3 (juniper_junos)
[15:24:36] [SUCCESS]: Reached Target.
  - Final Prompt: admin@vmx01>
--------------------------------------------------


# 実行とレポート表示

In [3]:
try:
    # --- 1. コマンドファイルの読み込み ---
    # param/commands.txt から実行するコマンドと「観点」「キーワード」を取得
    raw_lines, cmd_list = load_commands(CMD_FILE)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] コマンド実行開始... ({len(cmd_list)}件)")
    
    # --- 2. ネットワーク機器へのコマンド発行 ---
    # Netmikoを使用して全コマンドを順次実行し、結果を辞書形式で取得
    # read_timeout=300: 応答の重いコマンド（show log等）を考慮し5分に設定
    results = execute_commands(conn, cmd_list, read_timeout=300)
    
    # --- 3. HTMLレポート生成 (Jupyter画面用) ---
    # show_checklist=True: 画面上でポチポチ確認できるようサイドバーを表示
    html_for_display = generate_html_report(raw_lines, results, show_checklist=True)
    
    # --- 4. HTMLレポート生成 (ファイル保存用) ---
    # show_checklist=False: 印刷や証跡保存時に邪魔にならないようサイドバーを非表示にする
    html_for_save = generate_html_report(raw_lines, results, show_checklist=False)
    
    # --- 5. レポートのファイル出力 ---
    # result/フォルダ内に、ターゲット名と日付が入ったHTMLファイルとして書き出し
    with open(REPORT_FILE, "w", encoding="utf-8") as f:
        f.write(html_for_save)
    
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [SUCCESS]: レポートを保存しました: {REPORT_FILE}")
    
    # --- 6. Jupyter Notebook上へのレンダリング ---
    # 生成したHTMLを現在のセル直下に直接表示
    display(HTML(html_for_display))

except Exception as e:
    # 実行中のエラー（通信断など）やHTML生成時のエラーをキャッチ
    print(f"【エラー】: 実行またはレポート生成中にエラーが発生しました: {e}")

finally:
    # --- 7. セッションの確実な切断 ---
    # 正常終了でもエラー終了でも、最後に必ずSSH/Telnetセッションを切断してリソースを解放する
    if 'conn' in locals():
        conn.disconnect()
        print(f"[{datetime.now().strftime('%H:%M:%S')}] [INFO]: セッションを切断しました。")

[15:24:36] コマンド実行開始... (4件)
[15:24:37] [SUCCESS]: レポートを保存しました: /home/jupyter_projects/show-cmd-checker/result/report_clab-dev-vmx01_20260211.html


[15:24:37] [INFO]: セッションを切断しました。
